# JAX Quickstart

**JAX is NumPy on the CPU, GPU, and TPU, with great automatic differentiation for high-performance machine learning research.**

With its updated version of [Autograd](https://github.com/hips/autograd), JAX
can automatically differentiate native Python and NumPy code. It can
differentiate through a large subset of Python’s features, including loops, ifs,
recursion, and closures, and it can even take derivatives of derivatives of
derivatives. It supports reverse-mode as well as forward-mode differentiation, and the two can be composed arbitrarily
to any order.

What’s new is that JAX uses
[XLA](https://www.tensorflow.org/xla)
to compile and run your NumPy code on accelerators, like GPUs and TPUs.
Compilation happens under the hood by default, with library calls getting
just-in-time compiled and executed. But JAX even lets you just-in-time compile
your own Python functions into XLA-optimized kernels using a one-function API.
Compilation and automatic differentiation can be composed arbitrarily, so you
can express sophisticated algorithms and get maximal performance without having
to leave Python.


In [ ]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random

## Multiplying Matrices

We'll be generating random data in the following examples. One big difference between NumPy and JAX is how you generate random numbers. For more details, see the [README](../../README.md).

In [ ]:
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

Let's dive right in and multiply two big matrices.

In [ ]:
size = 3000
x = random.normal(key, (size, size), dtype=np.float32)
%timeit np.dot(x, x.T).block_until_ready()  # runs on the GPU

We added that `block_until_ready` because [JAX uses asynchronous execution by default](https://jax.readthedocs.io/en/latest/async_dispatch.html).

JAX NumPy functions work on regular NumPy arrays. 

In [ ]:
import numpy as onp  # original CPU-backed NumPy
x = onp.random.normal(size=(size, size)).astype(onp.float32)
%timeit np.dot(x, x.T).block_until_ready()

That's slower because it has to transfer data to the GPU every time. You can ensure that an NDArray is backed by device memory using `device_put`.

In [ ]:
from jax import device_put

x = onp.random.normal(size=(size, size)).astype(onp.float32)
x = device_put(x)
%timeit np.dot(x, x.T).block_until_ready()

The output of `device_put` still acts like an NDArray, but it only copies values back to the CPU when they're needed for printing, plotting, saving to disk, branching, etc. The behavior of `device_put` is equivalent to the function `jit(lambda x: x)`, but it's faster.

If you have a GPU (or TPU!) these calls run on the accelerator and have the potential to be much faster than on CPU.

In [ ]:
x = onp.random.normal(size=(size, size)).astype(onp.float32)
%timeit onp.dot(x, x.T)

JAX is much more than just a GPU-backed NumPy. It also comes with a few program transformations that are useful when writing numerical code. For now, there's three main ones:

 - `jit`, for speeding up your code
 - `grad`, for taking derivatives
 - `vmap`, for automatic vectorization or batching.

Let's go over these, one-by-one. We'll also end up composing these in interesting ways.

## Using `jit` to speed up functions

JAX runs transparently on the GPU (or CPU, if you don't have one, and TPU coming soon!). However, in the above example, JAX is dispatching kernels to the GPU one operation at a time. If we have a sequence of operations, we can use the `@jit` decorator to compile multiple operations together using [XLA](https://www.tensorflow.org/xla). Let's try that.

In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * np.where(x > 0, x, alpha * np.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

We can speed it up with `@jit`, which will jit-compile the first time `selu` is called and will be cached thereafter.

In [ ]:
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

## Taking derivatives with `grad`

In addition to evaluating numerical functions, we also want to transform them. One transformation is [automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation). In JAX, just like in [Autograd](https://github.com/HIPS/autograd), you can compute gradients with the `grad` function.

In [ ]:
def sum_logistic(x):
  return np.sum(1.0 / (1.0 + np.exp(-x)))

x_small = np.arange(3.)
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

Let's verify with finite differences that our result is correct.

In [ ]:
def first_finite_differences(f, x):
  eps = 1e-3
  return np.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                   for v in np.eye(len(x))])


print(first_finite_differences(sum_logistic, x_small))

Taking derivatives is as easy as calling `grad`. `grad` and `jit` compose and can be mixed arbitrarily. In the above example we jitted `sum_logistic` and then took its derivative. We can go further:

In [ ]:
print(grad(jit(grad(jit(grad(sum_logistic)))))(1.0))

For more advanced autodiff, you can use `jax.vjp` for reverse-mode vector-Jacobian products and `jax.jvp` for forward-mode Jacobian-vector products. The two can be composed arbitrarily with one another, and with other JAX transformations. Here's one way to compose them to make a function that efficiently computes full Hessian matrices:

In [ ]:
from jax import jacfwd, jacrev
def hessian(fun):
  return jit(jacfwd(jacrev(fun)))

## Auto-vectorization with `vmap`

JAX has one more transformation in its API that you might find useful: `vmap`, the vectorizing map. It has the familiar semantics of mapping a function along array axes, but instead of keeping the loop on the outside, it pushes the loop down into a function’s primitive operations for better performance. When composed with `jit`, it can be just as fast as adding the batch dimensions by hand.

We're going to work with a simple example, and promote matrix-vector products into matrix-matrix products using `vmap`. Although this is easy to do by hand in this specific case, the same technique can apply to more complicated functions.

In [ ]:
mat = random.normal(key, (150, 100))
batched_x = random.normal(key, (10, 100))

def apply_matrix(v):
  return np.dot(mat, v)

Given a function such as `apply_matrix`, we can loop over a batch dimension in Python, but usually the performance of doing so is poor.

In [ ]:
def naively_batched_apply_matrix(v_batched):
  return np.stack([apply_matrix(v) for v in v_batched])

print('Naively batched')
%timeit naively_batched_apply_matrix(batched_x).block_until_ready()

We know how to batch this operation manually. In this case, `np.dot` handles extra batch dimensions transparently.

In [ ]:
@jit
def batched_apply_matrix(v_batched):
  return np.dot(v_batched, mat.T)

print('Manually batched')
%timeit batched_apply_matrix(batched_x).block_until_ready()

However, suppose we had a more complicated function without batching support. We can use `vmap` to add batching support automatically.

In [ ]:
@jit
def vmap_batched_apply_matrix(v_batched):
  return vmap(apply_matrix)(v_batched)

print('Auto-vectorized with vmap')
%timeit vmap_batched_apply_matrix(batched_x).block_until_ready()

Of course, `vmap` can be arbitrarily composed with `jit`, `grad`, and any other JAX transformation.

## Parallelization with `pmap`

Suppose you want to run your computations on multiple XLA accelerators—GPUs or TPUs. JAX has an API for that called [pmap()](https://jax.readthedocs.io/en/latest/jax.html#jax.pmap) that lets you write single-program multiple-data (SPMD) programs. Google Cloud TPU pods are multi-host platforms and you can utilize them with this transform.

Similar to `jit`, when you apply `pmap` to a function, you compile with XLA. Then, you execute it in parallel on multiple devices.

Like `vmap`, this transform maps a function over array axes. But, unlike `vmap`, `pmap` replicates the function and executes each replica on its own XLA device in parallel. 

The transform also allows you to use all-reduce sum and other parallel SPMD collective operations.

### Enable Cloud TPUs in Google Colab

To take advantage of JAX's `pmap`, you'll be using Cloud TPUs in Google Colab:

- Change your Google Colab runtime by clicking **Edit** > **Notebook settings**
- Select **Hardware acceleration: TPU**
- If it says "Unable to connect to the runtime", click on the **Reconnect** button in the top right corner and wait until it has a green check mark (✓)

Then, run the cell below to further configure the TPU support:

In [ ]:
# Run this cell inside Google Colab with TPUs enabled
import requests
import os

# Import JAX libraries again, just in case (runtime switching resets the state)
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap, pmap, random

if 'TPU_DRIVER_MODE' not in globals():
  url = 'http://' + os.environ['COLAB_TPU_ADDR'].split(':')[0] + ':8475/requestversion/tpu_driver0.1-dev20191206'
  resp = requests.post(url)
  TPU_DRIVER_MODE = 1

# This is required to use TPU Driver as JAX's backend
from jax.config import config
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']
print(config.FLAGS.jax_backend_target)

Your output should have the Colab TPU IP address (gRPC) and port number.

- To list available devices for the backend, use [jax.devices()](https://jax.readthedocs.io/en/latest/jax.html#jax.devices):

In [ ]:
jax.devices

(The default backend is 'gpu' or 'tpu', if available. Otherwise it's 'cpu'.)

Let's go over some `pmap` examples.

- For a number of XLA devices available (e.g. 8), you can use `pmap` to map along a leading array axis:

In [ ]:
y = pmap(lambda x: x ** 2)(jnp.arange(8))

print(y)

- Run a dot product multiply on several devices:

In [ ]:
# Create a PRNG key and split it into 8 new ones
keys = random.split(random.PRNGKey(0), 8)

# Complile a new 5000x5000 matrix with `pmap`
size = 5000
matrices = pmap(lambda key: random.normal(key, (size, size)))(keys)

# Run the dot product of two matrices in parallel on XLA accelerators
result = pmap(jnp.dot)(matrices, matrices)

# Return the mean of the result and show the output
print(pmap(jnp.mean)(result))

- Measure the compute time:

In [ ]:
timeit -n 5 -r 5 pmap(jnp.dot)(matrices, matrices).block_until_ready()

- Compose autodiff (`grad`) with `pmap`:

In [ ]:
# Define an arbitrary array and reshape it
x = jnp.arange(8.).reshape((4, 2))

# Apply the @pmap decorator to run calculations across devices
@pmap
def f(x):
  y = jnp.sin(x)
  @pmap
  def g(z):
    return jnp.cos(z) * jnp.tan(y.sum()) * jnp.tanh(x).sum()
  # Call `grad`!
  return grad(lambda w: jnp.sum(g(w)))(x)

f(x)

The code here is simple. For a neural network example where you can do some data-parallel neural network training, check out [the SPMD MNIST example](https://github.com/google/jax/blob/master/examples/spmd_mnist_classifier_fromscratch.py) or the much more capable [Trax library](https://github.com/google/trax/).

This is just a taste of what JAX can do. We're really excited to see what you do with it!